**Task:** Clustering on UCI seed dataset, which can be downloaded from https://archive.ics.uci.edu/ml/datasets/seeds. The number of clusters is set as 3.

1. Implement K-means and GMM-EM algorithms from scratch (i.e., no third-party or off-the-shelf package or library are allowed). Explain briefly your source codes in the report.

**K-means** algorithm

In [238]:
import math
import random

f = open("seeds_dataset.txt", "r")
data = []

# data shape should be (210, 8) after read
for line in f:
    data.append([float(number) for number in line.split('\t') if number != ''])

data = [sample[:-1] for sample in data]
    
def cluster(data, K, is_random = False):
    n = len(data)
    d = len(data[0])
    feature_range = [(min([data[i][feature] for i in range(n)]),
                      max([data[i][feature] for i in range(n)])) for feature in range(d)]
    max_distance = sum([(feature_range[feature][1] - feature_range[feature][0]) ** 2 for feature in range(d)])

    c = [] # cluster centers
    r = [-1 for i in range(n)] # assignments
    
    # Initialization: randomize K cluster centers c
    c = [data[50 * k - 25] for k in range(K)]
    if is_random == True:
        c = random.sample(data, K)

    number_of_iterations = 0
    converged = False
    while not converged:
        
        # Assignment
        for i in range(n):
            distance = max_distance
            for k in range(K):
                current_distance = sum([(data[i][feature] - c[k][feature]) ** 2 for feature in range(d)])
                if current_distance < distance:
                    distance = current_distance
                    r[i] = k
                    
        # Refitting
        clusters = [[data[i] for i in range(n) if r[i] == k] for k in range(K)]
        
        new_c = [[sum([clusters[k][i][feature] for i in range(len(clusters))]) / len(clusters) for feature in range(d)] if len(clusters[k]) > 0 else c[k] for k in range(K)]

        if sum([abs(math.sqrt(sum([c[k][feature] ** 2 for feature in range(d)])) - math.sqrt(sum([new_c[k][feature] ** 2 for feature in range(d)]))) for k in range(K) if len(clusters[k]) > 0]) < 10 ** -6:
            converged = True

        c = [new_c[k] for k in range(K)]
        number_of_iterations += 1

    objective_value = sum([sum([(data[i][feature] - c[r[i]][feature]) ** 2 for feature in range(d)]) for i in range(n)])
        
    return clusters, objective_value, c, r, number_of_iterations
        
trials = 30
ans = cluster(data, 3)
for trial in range(trials - 1):
    clusters, objective_value, c, r, number_of_iterations = cluster(data, 3)
    if objective_value < ans[1]:
        ans = [clusters, objective_value, c, r, number_of_iterations]

print("Picking up the lowest obtained objective value from", trials, "trial(s), we have the cluster sizes as", [len(ans[0][k]) for k in range(len(ans[0]))], "with", number_of_iterations, "iterations")

Picking up the lowest obtained objective value from 30 trial(s), we have the cluster sizes as [57, 65, 88] with 4 iterations


**GMM-EM** algorithm

In [239]:
import math
import numpy as np
import random
import scipy.stats

f = open("seeds_dataset.txt", "r")
data = []

# data shape should be (210, 8) after read
for line in f:
    data.append([float(number) for number in line.split('\t') if number != ''])

data = [sample[:-1] for sample in data]
    
def GMM(data, K, is_random = False):
    n = len(data)
    d = len(data[0])
    feature_range = [(min([data[i][feature] for i in range(n)]),
                      max([data[i][feature] for i in range(n)])) for feature in range(d)]
    
    r = []
    
    mu = np.array([data[k] for k in range(K)]) # mean
    if is_random == True:
        mu = np.array(random.sample(data, K)) # mean
    cov = np.array([np.cov(np.transpose(data)) for k in range(K)]) # covariacne
    pi = np.array([1 / K for k in range(K)]) # mixing coefficients
    
    responsibilities = np.array([])
    
    def multivariate(x, mu, cov, custom = False):
        if custom and np.linalg.det(cov) != 0:
            pdf = 1

            pdf *= 1 / ((2 * math.pi) ** (d / 2))
            pdf *= np.linalg.det(cov) ** (-1/2)
            pdf *= math.e ** (-1/2 * np.matmul(np.matmul(np.transpose(x - mu), np.linalg.inv(cov)), (x - mu)))
        
            return pdf
        else:
            return scipy.stats.multivariate_normal(mu, cov, allow_singular = True).pdf(x)
    
    number_of_iterations = 0
    converged = False
    log_likelihood = None

    while not converged and number_of_iterations < 30:
        
        # E-step: Evaluate the responsibilities given current parameters
        responsibilities = [[] for i in range(n)]
        N = [0 for k in range(K)]
        for i in range(n):
            total = 0
            for k in range(K):
                responsibilities[i].append(pi[k] * multivariate(data[i], mu[k], cov[k]))
                total += responsibilities[i][k]
            for k in range(K):
                responsibilities[i][k] /= total
                N[k] += responsibilities[i][k]
                
        # M-step: Re-estimate the parameters given current responsibilities
        for k in range(K):
            mu[k] = np.multiply(responsibilities[0][k], data[0])
            for i in range(1, n):
                mu[k] = np.add(mu[k], np.multiply(responsibilities[i][k], data[i]))
            mu[k] = np.multiply(1 / N[k], mu[k])        
        for k in range(K):
            cov_total = np.full(shape = (d, d), fill_value = 0.0)
            for i in range(n):
                cov_total += responsibilities[i][k] * np.outer((data[i] - mu[k]), (data[i] - mu[k]))
            cov[k] = cov_total / N[k]
        for k in range(K):
            pi[k] = N[k] / n

        new_log_likelihood = sum([math.log(sum([pi[k] * multivariate(data[i], mu[k], cov[k]) for k in range(K)])) for i in range(n)])
        
        if log_likelihood != None and abs(new_log_likelihood - log_likelihood) < 10 ** -3:
            converged = True
        
        log_likelihood = new_log_likelihood
        r = [np.argmax(responsibilities[i]) for i in range(n)]
        
        number_of_iterations += 1
        if number_of_iterations % 5 == 0 and is_random == False:
            print("Current number of iterations:", number_of_iterations, "(max number of iterations is set to 30)")
    
    return mu, cov, pi, r, number_of_iterations
    
mu, cov, pi, r, number_of_iterations = GMM(data, 3)
print("From a total of", number_of_iterations, "iterations, we have the approximation of labeled data sizes as", [round(size) for size in (pi * len(data))])

Current number of iterations: 5 (max number of iterations is set to 30)
Current number of iterations: 10 (max number of iterations is set to 30)
Current number of iterations: 15 (max number of iterations is set to 30)
Current number of iterations: 20 (max number of iterations is set to 30)
Current number of iterations: 25 (max number of iterations is set to 30)
Current number of iterations: 30 (max number of iterations is set to 30)
From a total of 30 iterations, we have the approximation of labeled data sizes as [67, 60, 83]


2. Implement 2 evaluation metrics including Silhouette Coefficient and Rand Index from scratch (i.e., not calling off-the-shelf package) to evaluate the performance of above clustering algorithms.

In [240]:
def silhouette_coefficient(x, c, r, K):
    assigned = [[] for k in range(K)]
    for i in range(len(r)):
        assigned[r[i]].append(i)

    silhouette_coefficients = []
    for k in range(K):
        for i in range(len(assigned[k])):
            dist = None
            nearest_k = None
            for next_k in range(K):
                if next_k != k and (dist == None or dist < np.linalg.norm(c[next_k] - x[assigned[k][i]])):
                    dist = np.linalg.norm(c[next_k] - x[assigned[k][i]])
                    nearest_k = next_k
                
            intra_cluster_distances = []
            nearest_cluster_distances = []
            
            for j in range(len(assigned[k])):
                if i != j: intra_cluster_distances.append(np.linalg.norm(x[assigned[k][i]] - x[assigned[k][j]]))
            for j in range(len(assigned[nearest_k])):
                nearest_cluster_distances.append(np.linalg.norm(x[assigned[k][i]] - x[assigned[nearest_k][j]]))
            
            a = sum(intra_cluster_distances) / len(intra_cluster_distances)
            b = sum(nearest_cluster_distances) / len(nearest_cluster_distances)
            silhouette_coefficients.append((b - a) / max(a, b))
        
    return sum(silhouette_coefficients) / len(silhouette_coefficients)

def rand_index(y, y_expected):
    if len(y) != len(y_expected):
        return -1
    n = len(y)
    cnt = 0
    for i in range(n):
        for j in range(i + 1, n):
            if (y[i] == y[j] and y_expected[i] == y_expected[j]) or (y[i] != y[j] and y_expected[i] != y_expected[j]):
                cnt += 1
    return cnt / (n * (n - 1) / 2)

f = open("seeds_dataset.txt", "r")
data = []

# data shape should be (210, 8) after read
for line in f:
    data.append([float(number) for number in line.split('\t') if number != ''])        

x = [sample[:-1] for sample in data]
y = [int(sample[-1:][0]) for sample in data]

x = np.array(x) 
print("Silhouette Coefficient for K-means:", silhouette_coefficient(x, ans[2], ans[3], 3))
print("Silhouette Coefficient for GMM-EM:", silhouette_coefficient(x, mu, r, 3))
print("Rand Index for K-means:", rand_index(ans[3], y))
print("Rand Index for GMM-EM:", rand_index(r, y))

Silhouette Coefficient for K-means: 0.4859565093552149
Silhouette Coefficient for GMM-EM: 0.6705752238105781
Rand Index for K-means: 0.638049669628617
Rand Index for GMM-EM: 0.8864889496468444


3. Analyze the sensitivity to the initialization of each algorithm (e.g., run one clustering algorithm with random initialization multiple times, and calculate the standard deviations of evaluation scores of these clustering results) 

In [241]:
import statistics

f = open("seeds_dataset.txt", "r")
data = []

# data shape should be (210, 8) after read
for line in f:
    data.append([float(number) for number in line.split('\t') if number != ''])

x = [sample[:-1] for sample in data]
y = [int(sample[-1:][0]) for sample in data]
data = [sample[:-1] for sample in data]

trials = 10

silhouette_coefficient_results = []
rand_index_results = []
for trial in range(trials):
    ans = cluster(data, 3, is_random = True)
    silhouette_coefficient_results.append(silhouette_coefficient(np.array(x), ans[2], ans[3], 3))
    rand_index_results.append(rand_index(ans[3], y))
    
print("Standard Deviation of Silhouette Coefficient for K-means:", statistics.stdev(silhouette_coefficient_results))
print("Standard Deviation of Rand Index for K-means:", statistics.stdev(rand_index_results))


trials = 5   

silhouette_coefficient_results = []
rand_index_results = []
for trial in range(trials):
    mu, cov, pi, r, number_of_iterations = GMM(data, 3, is_random = True)
    silhouette_coefficient_results.append(silhouette_coefficient(np.array(x), mu, r, 3))
    rand_index_results.append(rand_index(r, y))    
    
print("Standard Deviation of Silhouette Coefficient for GMM-EM:", statistics.stdev(silhouette_coefficient_results))
print("Standard Deviation of Rand Index for GMM-EM:", statistics.stdev(rand_index_results))

Standard Deviation of Silhouette Coefficient for K-means: 0.038174147903931574
Standard Deviation of Rand Index for K-means: 0.04788233824981838
Standard Deviation of Silhouette Coefficient for GMM-EM: 0.16652724813904116
Standard Deviation of Rand Index for GMM-EM: 0.10331931286708596
